In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 전체 문장에서 약물 추출하기

In [3]:
def get_total_drugs():
    cur = conn.cursor()
#     cur.execute("""
#     SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
#     FROM
#     (SELECT id, tt_drug
#     FROM article_table_sentences_m M,
#         UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
#     WHERE (tt_drug='') is false) as d;
#     """)

# topic num 지정됨
    cur.execute("""
    SELECT id, TRIM(lower(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM 
        (SELECT * 
        FROM article_table_sentences_m
        WHERE table_id IN 
        (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

def get_drug_dictionary():
    cur = conn.cursor()
    cur.execute("""
    select cui_id, trim(lower(val)) from (
    (select cui1 as cui_id, cui1_str as val from dict_collapsed_final)
    UNION
    (select cui2 as cui_id, cui2_str as val from dict_collapsed_final)
    ) final;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['sent_id', 'name']

In [7]:
drugs['refined'] = ''
drugs.head()

,sent_id,name,refined
0,1531580,glucose,
1,1531580,lipoprotein cholesterol,
2,1531580,pravastatin,
3,2623936,interferon beta-1b,
4,1567608,digoxin,


#### 전처리 전 총 약물의 수는 137963개 입니다

In [8]:
drugs.shape

(137963, 3)

### 2. 전처리

A. dictionary를 참고하여, 정확히 일치하는 약물명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_drug_dictionary())
dictionary.columns = ['cui_id', 't_drug']
dictionary.drop_duplicates(inplace=True)

In [10]:
len(dictionary)

23931

In [11]:
def apply_dictionary(_drugs):
    for ind, dr in _drugs[_drugs['refined'] == ''].iterrows():
    #     print(ad['name'].strip())
        name = dr['name'].strip()
        exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)>0:
#             print(name)
            _drugs.loc[ind, 'refined'] = name
#             print(_drugs.loc[ind, 'refined'])
    return _drugs

drugs = apply_dictionary(drugs)

In [12]:
drugs[drugs.refined.str.strip() != ''].shape

(130180, 3)

#### B. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_hyj.xlsx파일을 통해 약물 이름이 아니라고 지정해주신 목록을 추가하였습니다.

In [13]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-facilitated', '-replacement', '-to-renin', '-gits', '-treated',\
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '&#8211',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      '-sham', '-adjusted', 'post-', '5m-', '20m-', '-provoked', '<3.2', '<3.3', '30-', '-synthase', '-responsive',\
                      '-pa-824', 'formulation-', 'mi-', 'discretion-', 'polychemotherapy', 'chemotherapy', 'seizures-','crack-',\
                      '-myocardial', 'exercise-', '—4803', '-low', 'mothers', 'von willebrand factor', 'others', '—1688', '-13',
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen', 'non-naproxen',\
                      'prothrombin', 'hypoalbuminaemia', 'multifood', 'epithelium', 'microalbuminuria', 'eosinophils',
                      'non-bevacizumab', 'albumin-creatinine', 'foods', 'non-cetuximab', 'eosinophilia', 'eosinophil', 'nonprotamine',
                      'albuminuria',
                      # 추가 약물이름이 아닌 경우
                      'post-food', 'tc-sulfur', 'iodine-to-creatinine', 'protein c2', 'tc–sulfur', 'blood-glucose', \
                      'cy5.5-arginine-glycine-aspartate', 'biotin-aacctgaccacagaggagaggc-3', 'n-2-mercaptopropionylglycine', \
                      'biotin-tccctcatccctgccatgt-3', 'animals-l-arg', 'biotin-cacggcacctggcctaaag-3', 'buserelin+cpax14',\
                      # 추가 확인 필요
                      'albumin:creatinine', 'glucose-insulin-potassium', 'oxygen-18', 'cyclooxygenase-2', '18f-fluorodeoxyglucose', 'cyclo-oxygenase'
                     ]

In [14]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    if row['refined']=='':
        refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
        if refined.strip() == '':
            drugs.drop(ind, inplace=True)
        elif row['name'].strip() != refined.strip():
            drugs.loc[ind, 'refined'] = refined
            print(row['name'])
            print(drugs.loc[ind, 'refined'])
        else:
            print(row['name'])
        print('===============================')

drugs = drugs.reset_index(drop=True)

lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
albumin-to-creatinine
ipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
lipoprotein cholesterol
epirubicin-cyclophosphamide
epirubicin-cyclophosphamide
epirubicin-cyclophosphamide
paclita

dobutamine-atropine
cy55-arginine-glycine-aspartate
cy55-arginine-glycine-aspartate
bivalirudinglycoprotein
heparinglycoprotein
peginterferon
peginterferon
peginterferon
radioiodine
etirinotecan
etirinotecan
etirinotecan
etirinotecan
etirinotecan
etirinotecan
etirinotecan
etirinotecan
etirinotecan
ketoprofene
ketoprofene
ketoprofene
mycophenolate
mycophenolate
rituximab-dose-dense
rituximab  
sorafenib-related
sorafenib 
sorafenib-related
sorafenib 
rituximab-dose-dense
rituximab  
rituximab-dose-dense
rituximab  
rituximab-dose-dense
rituximab  
teriflunomide14
rituximab-dose-dense
rituximab  
peg-interferon-alfa-2a
dolutegravir-rilpivirine
dolutegravir-rilpivirine
dolutegravir-rilpivirine
atovaquone-proguanil
salmeterolplacebo
salmeterolroflumilast
roflumilast-treated
roflumilast 
roflumilast-treated
roflumilast 
roflumilast-treated
roflumilast 
abacavir–lamivudine
atovaquone-proguanil
budesonide-formoterol
chloroquine-proguanil
rituximab-dose-dense
rituximab  
dobutamine-atropine
di

glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
g

5-fluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
irinotecanfluorouracil
irinotecanfluorouracil
oxaliplatinfluorouracil
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
darunavir-ritonavir
lopinavir-ritonavir
darunavir-ritonavir
lopinavir-riton

ganciclovir-treated
ganciclovir 
mycophenolate
ganciclovir-treated
ganciclovir 
fluorouracil-folinic
fluorouracil-folinic
trimethoprim-sulfamethoxazole
trimethoprim-sulfamethoxazole
trimethoprim-sulfamethoxazole
trimethoprim-sulfamethoxazole
pa-824-pyrazinamide
rifampicin-induced
rifampicin 
l-asparaginase
l-asparaginase
6-tioguanine
6-mercaptopurine
6-mercaptopurine
b3tcabacaviramprenavir
azt+3tc+abacavir+nfv
azt+3tc+abacavir+efv
3tcatazanavir
6-mercaptopurine
6-mercaptopurine
6-mercaptopurine
6-mercaptopurine
6-mercaptopurine
6-mercaptopurine
blockersdiazepam
cisplatin-based
cisplatin 
cisplatin-based
cisplatin 
pyrazinamide-ethambutol
pyrazinamide-ethambutol
pyrazinamide-ethambutol
pyrazinamide-ethambutol
levofloxacin-non-susceptible
levofloxacin 
levofloxacin-susceptible
levofloxacin 
levofloxacin-non-susceptible
levofloxacin 
levofloxacin-susceptible
levofloxacin 
levofloxacin-susceptible
levofloxacin 
levofloxacin-non-susceptible
levofloxacin 
levofloxacin-susceptible
levofloxaci

...

paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
sirolimus-eluting
sirolimus 
paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
paclitaxel-eluting
paclitaxel 
paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
glucose]
glucose 
nab-paclitaxel
nab-paclitaxel
nab-paclitaxel
nab-paclitaxel
sirolimus-eluting
sirolimus 
17-beta-estradiol
d-sotalol
dl-sotalol
d-sotalol
dl-sotalol
desethylamiodarone
irinotecan–temozolomide–temsirolimus
dolutegravir-rilpivirine
paclitaxel-eluting
paclitaxel 
miltefosineparomomycin
everolimus-eluting
everolimus 
paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
levodopa-carbidopa
levodopa-carbidopa
sirolimus-eluting
sirolimus 
sirolimus-eluting
sirolimus 
paclitaxel-coated
paclitaxel 
paclitaxel-eluting
paclitaxel 
paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
sirolimus-eluting
sirolimus 
paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
d-sotalol
sirolimus-eluting
sirolimus 
paclitaxel-eluting
pac

paclitaxel-eluting
paclitaxel 
sirolimus-eluting
sirolimus 
albumin-to-creatinine
quetiapine-er
quetiapine-er
piperacillin-tazobactammeropenem
artemether–lumefantrine
artesunate–amodiaquine
dihydroartemisinin–piperaquine
artemether–lumefantrine
pyronaridine–artesunate
pyronaridine–artesunate
cisplatin-based
cisplatin 
pyronaridine–artesunate
artemether–lumefantrine
artemether–lumefantrine
pyronaridine–artesunate
artemether–lumefantrine
artemether–lumefantrine
indinavir-containing
indinavir 
indinavir-containing
indinavir 
everolimus-eluting
everolimus 
levodopa-carbidopa
amodiaquine+sp
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
agelithium
malelithium
malelithium
sulphadoxine-pyrimethamine
sulphadoxine-pyrimethamine
sulphadoxine-pyrimethamine
amoxicillin-cefotaxime
trimethoprim-sulfamethoxazole
fluorouracil-based

nab-paclitaxel
nab-paclitaxel
nab-paclitaxel
nab-paclitaxel
amoxicillin-cefotaxime
amoxicillin-cefotaxime
sodium-glucose
mycophenolate
5-flucytosine
5-flucytosine
5-flucytosine
nab-paclitaxel
nab-paclitaxel
asparaginase-associated
asparaginase 
nab-paclitaxel
glucose-confirmed
glucose 
glucose-confirmed
glucose 
asparaginase-associated
asparaginase 
rituximab-dose-dense
rituximab  
carbamazepine-controlled
carbamazepine 
carbamazepine-controlled
carbamazepine 
5-fluorouracil
5-fluorouracil
sulphadoxine-pyrimethamine
sulphadoxine-pyrimethamine
sulphadoxine-pyrimethamine
sulphadoxine-pyrimethamine
anti-exenatide
methotrexate-naive
methotrexate 
carbamazepine-controlled
carbamazepine 
11c-d-threo-methylphenidate
11c-d-threo-methylphenidate
capecitabine-cisplatin
rituximab-dose-dense
rituximab  
glucose-confirmed
glucose 
glucose-confirmed
glucose 
11c-d-threo-methylphenidate
5-fluorouracil
5-fluorouracil
5-fluorouracil
5-fluorouracil
5-fluorouracil
capecitabine-cisplatin
capecitabine-cisp

formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
enoxaparin-idrabiotaparinux
levofloxacin-non-susceptible
levofloxacin 
levofloxacin-susceptible
levofloxacin 
formulation-budesonide
 budesonide
glucose-corrected
glucose 
glucose-corrected
glucose 
glucose-corrected
glucose 
lipoprotein cholesterol
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
formulation-budesonide
 budesonide
lopinavir-ritonavir
lopinavir-ritonavir
lopinavir-ritonavir
lopinavir-ritonavir
capecitabine-cisplatin
progesterone-receptor
irinotecan-based
irinotecan 
chemotherapycetuximab
 cetuximab
chemotherapycetuximab
 cetuximab
chemo

carboplatin-based
carboplatin 
abacavir–lamivudine
abacavir–lamivudine
abacavir–lamivudine
acarbose–metformin
acarbose–metformin
lipoprotein cholesterol
amphotericin bmiltefosine
bmiltefosine
amphotericin bparomomycin
bparomomycin
amphotericin bparomomycin
bparomomycin
amphotericin bmiltefosine
bmiltefosine
amphotericin bparomomycin
bparomomycin
amphotericin bmiltefosine
bmiltefosine
quetiapine-er
artemether–lumefantrine
artemether–lumefantrine
artemether–lumefantrine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
artesunate-mefloquine
dihydroartemisinin-piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
artesunate-mefloquine
dihydroartemisinin-piperaquine
artesunate-mefloquine
dihydroartemisinin-piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaqu

#### 전처리와 약물이 아닌 레코드를 제거하면 137524개가 남습니다

In [15]:
drugs.shape

(137524, 3)

## 복합 약물 정제

### 선생님께서 주신 엑셀 파일을 기반으로 정제합니다

In [16]:
replace_excel = pd.read_excel('./need_to_refined_hyj.xlsx', header=None)
replace_excel.columns = ['id', 'key', 'replace_with', '-', '-', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0,17-beta-estradiol,17-beta-estradiol
1,17-beta-estradiol-sham,17-beta-estradiol
2,17-hydroxyprogesterone,17-hydroxyprogesterone
3,5-flucytosine,5-flucytosine
4,5-hydroxyindoleacetic acid,5-hydroxyindoleacetic acid


In [17]:
replace_excel.tail()

,key,replace_with
452,aspirindipyridamole,aspirin
453,iodinebakechi,iodine
454,carvedilolmetoprolol,carvedilol + metoprolol
455,mgpotassiumdoxazosin,doxazosin
456,mgbendroflumethiazide,bendroflumethiazide


In [18]:
replace_excel.shape

(453, 2)

In [19]:
# replace_excel[replace_excel.key.str.find('digoxin')>=0]
#drugs[drugs.name.str.find('digoxin')>=0]

In [20]:
def replace_with_manual_curation(drugs):
    for ind, row in replace_excel.iterrows():
#         if len(drugs.loc[drugs.name==row['key']])>0:
#             print(row['key'])
#             print('==================')
        for d_ind, d_row in drugs.loc[drugs.name==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'])
            drugs.loc[d_ind, 'refined'] = origin_str 
#             print(d_row)
    return drugs

In [21]:
drugs = replace_with_manual_curation(drugs)

In [22]:
# drugs[drugs.name.str.find('digoxin')>=0]

In [23]:
drugs[drugs.name.str.find('carvedilolmetoprolol')>=0][:2]

,sent_id,name,refined
17166,1421397,carvedilolmetoprolol,carvedilol + metoprolol
17168,1421404,carvedilolmetoprolol,carvedilol + metoprolol


In [24]:
drugs[drugs.refined.str.find('zidovudine lamivudine amprenavir')>=0]

,sent_id,name,refined
92158,492550,azt3tcamprenavir,zidovudine lamivudine amprenavir


## 아직 추가 전처리가 필요한 약물 수

In [25]:
drugs[drugs['refined'] == ''].shape

(1063, 3)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

### 선생님께서 피드백 주신데로 non은 제외하였습니다.
### 선생님께서 피드백 주신데로 약물명 전처리에서 collagen, blood, glucose, oxygen, nitrogen-13 등을 추가하였습니다.

In [26]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting',
                  '-prev', '-insensitive', 'resistant', 'refractory', 'pre-', 'lipoprotein', '-untreated', 'anti-',
                  '-receptor', '-tolerance', '-intolerant', 'non-secretin', 'collagen', 'blood', 'glucose', 'oxygen',\
                  'nitrogen-13', 'thrombin-antithrombin', 'fetoprotein', 'renin', 'cholesterol', 'albumin:creatinine']

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [27]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
#         print(row['name'])
        drugs.drop(ind, inplace=True)
#         print('===============================')

drugs = drugs.reset_index(drop=True)

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 1336167남습니다

In [28]:
drugs.shape

(133617, 3)

### 여러 약물이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [29]:
for ind, dr in drugs[(drugs.name.str.find('-')>=0) & (drugs['refined'] == '')].iterrows():
    if dr['name'].find('-to-')>=0:
#         dr['refined'] = dr['name'].replace('-to-', '+')
        drugs.loc[ind, 'refined'] = dr['name'].replace('-to-', ' + ')

In [30]:
drugs[drugs.name.str.find('+')>=0]

,sent_id,name,refined
12383,492349,azt+3tc+abacavir+nfv,zidovudine + lamivudine + abacavir + nelfinavir
12385,492353,azt+3tc+abacavir+efv,zidovudine lamivudine abacavir efavirenz
31976,2039780,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine
31981,2039786,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine
38072,177028,tiotropium+roflumilast,tiotropium + roflumilast
57166,177022,tiotropium+roflumilast,tiotropium + roflumilast
57196,2039798,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine
58146,2039810,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine
62363,177016,tiotropium+roflumilast,tiotropium + roflumilast
67133,2039816,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine


In [31]:
def separate_unique_drugs(drugs):
    additional_drugs = []

    for ind, row in drugs[drugs.refined.str.find('+')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            drugs.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:
                additional_drugs.append({'sent_id':row['sent_id'], 'name':row['name'], 'refined':pro})

    for ind, row in drugs[drugs.refined.str.find(',')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            drugs.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_drugs.append({'sent_id':row['sent_id'], 'name':row['name'], 'refined':pro})


    drugs = drugs.append(pd.DataFrame(additional_drugs))
    drugs.drop_duplicates(inplace=True)
    return drugs

drugs = separate_unique_drugs(drugs)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


#### 식별 가능한 약물명을 나누고 나면 137014개가 됩니다.

In [32]:
drugs.shape

(137014, 3)

In [33]:
drugs[drugs.name.str.find('taxane-cyclophosphamide')>=0]

,name,refined,sent_id
1285,taxane-cyclophosphamide,taxane,2456322
37923,mono-taxane-cyclophosphamide,taxane,2456320
39,taxane-cyclophosphamide,cyclophosphamide,2456322
991,mono-taxane-cyclophosphamide,cyclophosphamide,2456320


### 정제후 다시 한번 dictionary와 매칭

In [34]:
drugs = apply_dictionary(drugs)

### 앞 뒤의 -, . 제거

In [35]:
remove_list = ['.', '-', ':', '+', '–', ';', '%']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
drugs.name = drugs.name.map(lambda x: preprocess(x))

In [36]:
_drugs = replace_with_manual_curation(drugs.loc[drugs.refined==''])
_drugs

/home/grace/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,name,refined,sent_id
4731,acetylcholinesterase,,468608
54778,chloroquine-proguanil,chloroquine + proguanil,647917
60708,tetracyclines,,3547
64079,interferon-gamma,interferon-gamma,2237034
91228,tetracyclines,,3548
92926,interferon-gamma,interferon-gamma,2237033
99587,acetylcholinesterase,,468606
104410,interferon-gamma,interferon-gamma,2237035
104486,acetylcholinesterase,,468605
129860,acetylcholinesterase,,468607


In [37]:
for ind, row in _drugs.iterrows():
    if row['refined'].strip() != '':
        drugs.loc[ind, 'refined'] = row['refined']

In [38]:
# drugs.loc[drugs.name=='digoxin']
# drugs.loc[drugs.refined=='epirubicin + cyclophosphamide']

In [39]:
drugs = separate_unique_drugs(drugs)

In [40]:
drugs[(drugs['refined'] == '')]

,name,refined,sent_id
4731,acetylcholinesterase,,468608
60708,tetracyclines,,3547
91228,tetracyclines,,3548
99587,acetylcholinesterase,,468606
104486,acetylcholinesterase,,468605
129860,acetylcholinesterase,,468607


In [41]:
for ind, dr in drugs[(drugs['refined'] == '')].iterrows():
    found_in_excel = replace_excel[replace_excel.key.str.find(dr['name'])>=0]
    if len(found_in_excel)>0:
        print(dr['name'])
        print(found_in_excel)

In [42]:
# drugs[drugs.name.str.find('digoxin')>=0]
# drugs[drugs.sent_id==1567608]

## 정체된 목록입니다.

In [43]:
len(drugs.sent_id.unique())

96931

In [44]:
dictionary.head()

,cui_id,t_drug
0,C0000266,parlodel
1,C0000294,mesna
2,C0000378,dops
3,C0000378,droxidopa
4,C0000378,threo dops


In [ ]:
drugs['drug_id'] = durgs.refined.apply(lambda x: )

In [ ]:
for ind, grp in drugs.groupby('sent_id'):
    print(ind)
    print(grp['refined'].unique())

### 추가 확인 요청 데이터

In [ ]:
drugs[drugs['refined'] == '']

In [ ]:
drugs[drugs['refined'] == ''].to_csv('./need_to_refined_0225.csv', sep=',')

In [ ]:
drugs.describe()